# 1. Introduction

***Nama         :*** Muhammad Fiqih Al-ayubi

***Batch        :*** HCK-017

***Objective    :*** 

Program ini dibuat untuk memvalidasi dataset hasil cleaning dan menyimpan konfigurasi validasi agar proses validasi dapat dilakukan lagi dengan dataset yang telah diupdate atau dataset baru (dengan konteks problem yang sama). Selain itu, program ini juga dibuat untuk memastikan jika dataset hasil cleaning siap digunakan untuk proses analisa atau proses - proses lain yang membutuhkan perhitungan matematis


# 2. Import Libraries

In [1]:
from great_expectations.data_context import FileDataContext
import pandas as pd

# 3. Mendefinisikan Aset dan Datasource

In [2]:
# Membuat context
context = FileDataContext.create(project_root_dir='./')

In [3]:
# Membuat nama datasource
datasource_name = 'datasource_m3'

# Conditional statement untuk menghapus datasource dengan nama yang sama
if context.datasources: 
    context.delete_datasource(datasource_name)
datasource = context.sources.add_pandas(datasource_name)

In [4]:
# Membuat nama data asset
asset_name = 'asset_m3'

# Membuat file untuk mengakses data asset
path_to_data = '/Users/Fiqih/AppData/Local/Programs/Latihan python/hacktiv8/Milestone/Milestone3/Latihan/Milestone3/dags/P2M3_fiqih_data_clean.csv'

# Membuat aset data berdasarkan file yang telah ditentukan path nya
asset = datasource.add_csv_asset(name=asset_name, filepath_or_buffer=path_to_data)

In [5]:
# Membuat permintaan data dari aset yang telah ditentukan
batch_request = asset.build_batch_request()

# 4. Expectations Suite

Berikut adalah beberapa persyaratan wajib yang harus dimiliki dataset hasil cleaning :

***1. Seluruh nilai pada kolom travel_id harus bersifat unique***

Syarat ini untuk memastikan jika tidak ada kesalahan dalam proses pengumpulan data. Jika travel_id memiliki 2 atau lebih nilai yang sama, itu artinya terdapat kesalahan dalam pengumpulan data karena tidak mungkin agensi mengeluarkan lebih dari 1 travel id yang sama. Tidak terpenuhinya syarat ini juga dapat mengindikasikan jika proses celaning data belum maksimal


***2. Nilai pada kolom commision harus berada pada rentang tertentu (expect column values to be between)***

Kegagalan dalam syarat ini dapat menyebabkan analisa keuntungan agensi asuransi menjadi bias karena net sales atau keuntungan bersih agensi asuransi juga dipengaruhi besarnya komisi yang diberikan kepada agennya. Oleh karena itu, nilai komisi haruslah realistis atau sesuai dengan hasil cleaning


***3. Kolom claim harus memiliki nilai antara 'Yes' atau 'No' (expect column values to be in set)***

Kegagalan dalam persyaratan ini mengindikasikan adanya kesalahan fatal dalam proses data cleaning. Hal ini dikarenakan nilai pada kolom claim menjadi parameter utama dalam melakukan proses analisa dan klasifikasi customer


***4. Kolom net_sales harus memiliki data numerik atau numerik float (expect_column_values_to_be_in_type_list)***

Memastikan tipe data kolom net_sales setelah proses pembersihan adalah hal yang wajib untuk mengantisipasi jika data yang telah dibersihkan akan digunakan lagi untuk perhitungan statistik atau pelatihan model machine learning


***5. Nilai unique pada kolom destination harus memiliki proporsi pada rentang tertentu terhadap keseluruhan data (expect column proportion of unique values to be between)***

Ini artinya, varians nilai pada kolom destination harus lebih kecil dibandingkan jumlah baris data. Dalam hal ini maximal proportion dihitung berdasarkan total jumlah negara yang eksis lalu dibagi dengan total jumlah baris. Jika proporsi melebihi jumlah maksimal itu artinya terdapat kesalahan dalam penginputan nama destinasi/negara


***6. Jumlah baris data harus sesuai dengan hasil data cleaning (expect table row count to be between)***

Mengingat adanya missing values dan duplicated rows pada dataset raw, maka jumlah baris data harus sama speerti jumlah baris setelah proses cleaning. Selain memvalidasi jumlah baris data, proses ini juga bertujuan untuk memberikan batasan minimum baris yang diperlukan untuk kebutuhan analisa karena baris/entries yang terlalu sedikit tidak cukup untuk menghasilkan kesimpulan yang menggambarkan pengaruh dari masing - masing independent variable


***7. Nama dan jumlah kolom harus sesuai dengan hasil proses cleaning (expect_table_columns_to_match_set)****

Artinya baik format nama maupun jumlah kolomnya harus sama seperti data hasil proses cleaning. Jika nama dan jumlah kolom tidak sama dengan hasil yang didapatkan setelah proses cleaning, itu artinya terdapat kesalahan dalam proses penyimpanan data atau data yang diinput pada proses validasi merupakan data yang salah




In [6]:
# Membuat expectation suite
expectation_suite_name = 'expectation_data_m3'
context.add_or_update_expectation_suite(expectation_suite_name)

# Membuat validator berdasarkan data yang telah direquest dengan batch request
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Cek hasil
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,agency,agency_type,distribution_channel,product_name,claim,duration,destination,net_sales,commision,gender,age,travel_id
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81,e7d9ccc6-321c-4986-a102-82aa50ba12f6
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71,c35899e3-4252-40bc-817d-965fb977cc42
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,Unknown,32,484f3c54-1f50-414f-ab7b-b021f65b88ff
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,Unknown,32,00222938-6f40-423c-bfd7-09024e781c69
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,Unknown,41,0cabd629-ab5f-4c99-88df-2078abcd9d86


# 5. Data Validation

### 5.1. Expectation 1 : Seluruh nilai pada kolom travel_id harus unique

In [7]:
# Proses validasi
validator.expect_column_values_to_be_unique('travel_id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 55284,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dari 55284 baris data kolom travel_id yang divalidasi keunikannya, hasilnya seluruh nilai pada kolom travel_id bersifat unique sehingga hasil validasi berstatus true. 

### 5.2. Expectation 2 : Nilai pada kolom commision harus berada pada rentang tertentu

Pada proses ini, nilai minimal akan diset sebesar 0 dan nilai maksimal akan diset sesuai dengan nilai maksimal dari kolom commision yang dihasilkan dari proses data cleaning (283.5)

In [8]:
# Proses validasi
validator.expect_column_values_to_be_between(column='commision', min_value=0, max_value=283.5)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 55284,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dari hasil proses validasi di atas yang berstatus true, maka dapat disimpulkan jika nilai commision telah memenuhi syarat berada pada range nilai minimum dan nilai maksimum.

### 5.3. Expectation 3 : Kolom claim hanya memiliki nilai 'Yes' atau 'No'

In [9]:
# Memasukan nilai unique kolom claim ke dalam list
claim_values = ['Yes', 'No']

# Proses validasi
validator.expect_column_values_to_be_in_set('claim', claim_values)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 55284,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Karena hasil validasi di atas berstatus true, maka dapat disimpulkan jika nilai pada kolom 'claim' telah memenuhi syarat dan tidak ada nilai lain pada kolom ini yang dapat membuat hasil analisa menjadi ambigu.

### 5.4. Expectation 4 : Kolom net_sales harus memiliki tipe data numerik integer atau numerik float

In [10]:
# Membuat list tipe data
type_list = ['integer', ['float']]

# Proses validasi
validator.expect_column_values_to_be_in_type_list('net_sales', ['integer', 'float'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Karena hasil validasi berstatus true maka dapat disimpulkan jika tipe data kolom net_sales sudah sesuai dengan apa yang disyaratkan dan nilai pada kolom tersebut dapat digunakan untuk perhitungan matematis pada proses - proses selanjutnya.

### 5.5. Expectation 5 : Nilai unique pada kolom destination harus memiliki proporsi pada rentang tertentu terhadap keseluruhan data

Nilai maksimal dihasilkan dari pembagian antara jumlah negara yang eksis dengan jumlah baris pada dataset hasil cleaning. Nilai minimal dihasilkan dari pembagian 1 negara (yaitu singapura karena seluruh agency berasal dari negara tersebut) dengan jumlah baris dataset hasil cleaning.

In [11]:
validator.expect_column_proportion_of_unique_values_to_be_between(
    column="destination",
    min_value=1.808841617827943e-05,
    max_value=0.0035
)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 0.002695174010563635
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Hasil validasi berstatus true, itu artinya proporsi nilai unique pada kolom destination/negara tujuan tidak melebihi jumlah negara yang eksis dan hasil tersebut sudah sesuai dengan apa yang dipersyaratkan.

### 5.6. Expectation 6 : Jumlah baris data harus sesuai dengan hasil data cleaning

In [12]:
# Proses validasi
validator.expect_table_row_count_to_be_between(min_value = 40000, max_value = 55284)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 55284
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dari hasil di atas maka dapat disimpulkan jika jumlah baris tidak melebihi apa yang telah dihasilkan dari proses cleaning dan jumlah baris tidak kurang dari apa yang dibutuhkan untuk proses analisa data.

### 5.7. Expectation 7 : Nama dan jumlah kolom harus sesuai dengan hasil proses cleaning

In [13]:
# Proses validasi
validator.expect_table_columns_to_match_set(column_set = ['agency', 'agency_type', 'distribution_channel', 'product_name', 'claim', 'duration', 'destination', 'net_sales', 'commision', 'gender', 'age', 'travel_id'])

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": [
      "agency",
      "agency_type",
      "distribution_channel",
      "product_name",
      "claim",
      "duration",
      "destination",
      "net_sales",
      "commision",
      "gender",
      "age",
      "travel_id"
    ]
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Dari hasil di atas maka dapat disimpulkan jika jumlah dan nama kolom tidak memiliki penyimpangan dengan dataset hasil cleaning. Itu artinya tidak ada kesalahan dalam proses penyimpanan data atau data yang diinput pada proses validasi merupakan data yang benar.

### 5.8. Menyimpan konfigurasi proses validasi

In [14]:
# Menyimpan konfigurasi
validator.save_expectation_suite(discard_failed_expectations=False)

# 6. Membuat Checkpoint

In [15]:
# Membuat check point
checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [16]:
# Menjalankan check point
checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/28 [00:00<?, ?it/s]

# 7. Membuat Dokumentasi Konfigurasi dan Hasil Validasi

In [17]:
# Membuat dokumentasi
context.build_data_docs()

{'local_site': 'file://c:\\Users\\Fiqih\\AppData\\Local\\Programs\\Latihan python\\hacktiv8\\Milestone\\Milestone3\\Latihan\\Milestone3\\gx\\uncommitted/data_docs/local_site/index.html'}

# 8. Kesimpulan
Berdasarkan proses validasi di atas maka dapat disimpulkan jika dataset hasil cleaning telah memenuhi persyaratan validasi dan dataset tersebut telah sesuai dengan hasil cleaning sehingga dataset tersebut dapat digunakan untuk proses analisa.